# Update and Delete via Joins

Once utilising the syntax of UPDATE and DELETE using table aliases it is a simple step to incorporate a JOIN into the mixture.

This has the benefit of allowing for more dynamic queries to be written and executed against a database that would otherwise be highly manual and invariably slow processes.

#### Update Via Join

In this example we will look back towards our previous UPDATE examples in which we used the UPDATE statement to allocated a salary increase for our employees.

However, this time we shall take this one step further.

Let's have a quick look at some key columns in our hr.employee table:

In [ ]:
select employeeID, title, firstName, lastName, salary, jobTitleID
from JupyterDatabase.hr.employee;

Let's also have a quick look at another, related table called hr.jobTitle.  Unsurprisingly, this holds all the JobTitle information to which the hr.employee table is linked via the jobTitleID column:

In [ ]:
select *
from JupyterDatabase.hr.jobTitle;

We can see that there are several jobTitles listed as having a 'JobGrade' of 6. These are the ones we will focus on for this example as we have been handed the following requirement:

Please increase the salary of all employees with jobGrade 6 by 10%.

Obviously we could do this manually by listing all jobTitleIDs with jobGrade 6 and then looking through our 200+ employees for anyone matching those jobTItleIDs, noting down those employeeIDs and then performing the UPDATE on that manual list of employeeIDs, but even this description alone sounds tiring, let alone performing the action itself.

We also know that, in SQL, we can easily obtain this information with a JOIN:

In [ ]:
select e.employeeID, e.title, e.firstName, e.lastName, e.salary, j.jobGrade
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

This tells us we have 80 employees affected by the proposed salary increase which, again, is not a pleasant volume to cater for manually, let alone if this were a much larger corporate database.

So let's use this query to calculate our salary increases:

In [ ]:
select e.salary, e.salary * 1.1
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

This is exactly what we need and will now form the basis of our demo.

However, before we can start, where this is the exact starting point we need for a live change we are making in our database tables, in this case we do not wish to affect the real tables within our database (so as to provide consistency to all using it) we need to move away from real tables and to create a temporary table (so as to allow our demos to run without interferring with real data or affect other users).

Let's create a temporary table for our employee information and then join that to hr.jobTitle in order to re-create the query output above. This will be our starting point going forwards:

In [ ]:
---------------------------------------------------------------------------
-- Creating a temporary table to mimic hr.employee
---------------------------------------------------------------------------
drop table if exists #employeeData;

create table #employeeData
(
    employeeID int,
    title varchar(5),
    firstName varchar(25),
    lastName varchar(35),
    salary decimal(10, 2),
    jobTitleID int
);

insert into #employeeData
(
    employeeID, title, firstName, lastName, salary, jobTitleID
)
select employeeID, title, firstName, lastName, salary, jobTitleID
from JupyterDatabase.hr.employee;
---------------------------------------------------------------------------

-- Now we can use #employeeData as if it were hr.employee
select e.salary, e.salary * 1.1
from #employeeData as e -- Note the use of our temporary table instead of hr.employee
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

This gives us the exact results we are looking for and therefore we can now move on to our UPDATE statement without fear of editing live data.

Hopefully you can already see how this might work based on the fuller syntax we introduced within the UPDATE section:

<code>UPDATE <i>tableAlias</i><br/>
SET <i>currentValueOrColumn</i> = <i>newValueOrColumn</i><br/>
FROM <i>myTable</i> AS <i>tableAlias</i><br/>
WHERE <i>column</i> = <i>value</i></code>

This is exactly why we introduced this style of UPDATE as it utilises table aliases and we do not change anything below FROM, simply swapping SELECT for our UPDATE and SET keywords.

Let's look at this more closely.

First we exchange our SELECT line for our UPDATE and SET:

<code>select e.salary, e.salary * 1.1</code>

As we are updating e.salary we are updating table alias 'e'.  Therefore this becomes:

<code>update e<br/>
set e.salary = e.salary * 1.1</code>

And the FROM portion remains the same irrespective:

<code>from #employeeData as e<br/>
join JupyterDatabase.hr.jobTitle as j<br/>
on e.jobTitleID = j.jobTitleID<br/>
where j.jobGrade = 6;</code>

Putting these together results in the following:

<code>update e<br/>
set e.salary = e.salary * 1.1<br/>
from #employeeData as e<br/>
join JupyterDatabase.hr.jobTitle as j<br/>
on e.jobTitleID = j.jobTitleID<br/>
where j.jobGrade = 6;</code>

This is exactly what we want and can test this by running the following full script:

In [ ]:
---------------------------------------------------------------------------
-- Creating a temporary table to mimic hr.employee
---------------------------------------------------------------------------
drop table if exists #employeeData;

create table #employeeData
(
    employeeID int,
    title varchar(5),
    firstName varchar(25),
    lastName varchar(35),
    salary decimal(10, 2),
    jobTitleID int
);

insert into #employeeData
(
    employeeID, title, firstName, lastName, salary, jobTitleID
)
select employeeID, title, firstName, lastName, salary, jobTitleID
from JupyterDatabase.hr.employee;

---------------------------------------------------------------------------
-- Check on our 'before' data

select e.employeeID, j.jobGrade, e.salary
from #employeeData as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

-- Perform the UPDATE via a JOIN
update e
set e.salary = e.salary * 1.1 -- SELECT becomes UPDATE and SET
from #employeeData as e       -- Remainder of query remains unchanged
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

-- Verify the results
select e.employeeID, j.jobGrade, e.salary
from #employeeData as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

As we can see from the above, once we are comfortable with table aliases then performing both a basic update or an update via a join are quite simple to understand and code.

#### Delete via Join

Likewise we saw that the DELETE statement was incredibly similar to the UPDATE statement in syntax and this remains the case with the JOIN based delete as well.

Once we understand table aliasing and the UPDATE via a JOIN syntax then DELETE follows in exactly the same manner.

We shall demonstrate this using an almost identical example as for UPDATE except that instead of updating all records with jobGrade 6 we shall DELETE them.

Let's do a quick count (remembering back to our Aggregates code) to display the number of records in our hr.employee table (in this case our temporary copy as we do not wish to edit live tables) and, using a JOIN, the number of those which have jobGrade 6:

In [ ]:
---------------------------------------------------------------------------
-- Creating a temporary table to mimic hr.employee
---------------------------------------------------------------------------
drop table if exists #employeeData;

create table #employeeData
(
    employeeID int,
    title varchar(5),
    firstName varchar(25),
    lastName varchar(35),
    salary decimal(10, 2),
    jobTitleID int
);

insert into #employeeData
(
    employeeID, title, firstName, lastName, salary, jobTitleID
)
select employeeID, title, firstName, lastName, salary, jobTitleID
from JupyterDatabase.hr.employee;

---------------------------------------------------------------------------

-- How many records in the table
select count(*)
from #employeeData

-- How many records with jobGrade = 6
select count(*)
from #employeeData as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

This tells us that we have 224 records in total, 80 of which are jobGrade 6.

Therefore if we were to DELETE all those records with jobGrade 6 we should obtain a result of 144 records remaining in our table.

Now let's consider our syntax based on the table alias DELETE we covered earlier:

<code>DELETE <i>tableAlias</i><br/>
FROM <i>myTable</i> AS <i>tableAlias</i><br/>
WHERE <i>column</i> = <i>value</i></code>

This syntax is even easier to move between SELECT and DELETE as we simply swap out our SELECT line for the keyword DELETE followed by the table alias of the table from which we wish to remove records.

Let's look at this more closely.

First we exchange our SELECT line for our DELETE:

<code>select *</code>

We are deleting from our #employeeData table, hence need the alias 'e'.  Therefore this becomes:

<code>delete e</code>

And the FROM portion remains the same irrespective:

<code>from #employeeData as e<br/>
join JupyterDatabase.hr.jobTitle as j<br/>
on e.jobTitleID = j.jobTitleID<br/>
where j.jobGrade = 6;</code>

Putting these together results in the following:

<code>delete e<br/>
from #employeeData as e<br/>
join JupyterDatabase.hr.jobTitle as j<br/>
on e.jobTitleID = j.jobTitleID<br/>
where j.jobGrade = 6;</code>

This is exactly what we want and can test this by running the following full script:

In [ ]:
---------------------------------------------------------------------------
-- Creating a temporary table to mimic hr.employee
---------------------------------------------------------------------------
drop table if exists #employeeData;

create table #employeeData
(
    employeeID int,
    title varchar(5),
    firstName varchar(25),
    lastName varchar(35),
    salary decimal(10, 2),
    jobTitleID int
);

insert into #employeeData
(
    employeeID, title, firstName, lastName, salary, jobTitleID
)
select employeeID, title, firstName, lastName, salary, jobTitleID
from JupyterDatabase.hr.employee;

---------------------------------------------------------------------------
-- Check on our 'before' data

select count(*) as numJobGrade6
from #employeeData as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

-- Perform the DELETE via a JOIN
delete e
from #employeeData as e       -- Remainder of query remains unchanged
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

-- Verify the results
select count(*) as numJobGrade6
from #employeeData as e
join JupyterDatabase.hr.jobTitle as j
on e.jobTitleID = j.jobTitleID
where j.jobGrade = 6;

-- Final count of #employeeData
select count(*) as countMinusJobGRade6
from #employeeData;

As with UPDATE, this syntax is clear and simple to use once we understand how to use a table alias within our code.

Armed with the table alias, we can easily identify the records we wish to DELETE and remove them accordingly.